# Labo 8 : modèles de langage / Cloze Test
Par Guidoux Vincent et Gonzalez Montes Nathan

## Objectif et plan

L'objectif de ce labo est d'entraîner des modèles de langues (EN : language models) sous NLTK en utilisant le package nltk.lm. Les modèles seront entraînés sur des romans de Balzac en français
(fournis par le projet Gutenberg), et leurs performances seront mesurées par leur perplexité sur de nouveaux textes. Les modèles serviront également à deviner des mots cachés dans un texte, et ici leurs performances seront comparées à celles des humains sur cette même tâche.

## Importation

In [1]:
import nltk
import nltk.lm
from nltk.lm.preprocessing import padded_everygram_pipeline
import os, codecs
from urllib import request
import random

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nortalle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Obtenir les données

Télécharger les dix romans de la Comédie humaine de Balzac en français, enlever les notices en anglais au début et à la fin, puis les assembler en un corpus.

In [2]:
def dl_gutenberg_raw(url):
    print("dowloading \"{}\" ... ".format(url))
    
    response = request.urlopen(url)
    raw = response.read().decode('utf8')
    
    # enlever les notices en anglais au début 
    begin = raw.find("Libraries)") + 10
    # et à la fin
    end = raw.find("*** END OF THIS PROJECT")
    
    return raw[begin:end]

In [3]:
volume_1 = "https://www.gutenberg.org/ebooks/41211.txt.utf-8"
volume_2 = "https://www.gutenberg.org/files/43851/43851-0.txt"
volume_3 = "https://www.gutenberg.org/files/45060/45060-0.txt"
volume_4 = "https://www.gutenberg.org/files/48082/48082-0.txt"
volume_5 = "https://www.gutenberg.org/files/49482/49482-0.txt"
volume_6 = "https://www.gutenberg.org/files/51381/51381-0.txt"
volume_7 = "https://www.gutenberg.org/files/52831/52831-0.txt"
volume_8 = "https://www.gutenberg.org/files/54723/54723-0.txt"
volume_9 = "https://www.gutenberg.org/files/55860/55860-0.txt"
volume_10 = "https://www.gutenberg.org/files/58244/58244-0.txt"

balzac = [volume_1, 
          volume_2, 
          volume_3, 
          volume_4, 
          volume_5, 
          volume_6, 
          volume_7, 
          volume_8,
          volume_9, 
          volume_10]

Utiliser une procédure à laquelle on donnera la liste de noms de fichiers ou des URLs.

In [4]:
corpus = ""

for volume_url in balzac:
    corpus += dl_gutenberg_raw(volume_url)

dowloading "https://www.gutenberg.org/ebooks/41211.txt.utf-8" ... 
dowloading "https://www.gutenberg.org/files/43851/43851-0.txt" ... 
dowloading "https://www.gutenberg.org/files/45060/45060-0.txt" ... 
dowloading "https://www.gutenberg.org/files/48082/48082-0.txt" ... 
dowloading "https://www.gutenberg.org/files/49482/49482-0.txt" ... 
dowloading "https://www.gutenberg.org/files/51381/51381-0.txt" ... 
dowloading "https://www.gutenberg.org/files/52831/52831-0.txt" ... 
dowloading "https://www.gutenberg.org/files/54723/54723-0.txt" ... 
dowloading "https://www.gutenberg.org/files/55860/55860-0.txt" ... 
dowloading "https://www.gutenberg.org/files/58244/58244-0.txt" ... 


In [5]:
print("Il y a {} caractères dans ce corpus.".format(len(corpus)))

Il y a 12214531 caractères dans ce corpus.


Sauvegarder le texte résultant : quelle taille fait-il en Ko ou Mo?

In [6]:
filename1 = "la_comedie_humaine.txt"

if os.path.exists(filename1): 
    os.remove(filename1)
fd = codecs.open(filename1, 'a', 'utf8')

try:
    fd.write(corpus)
    

finally:  
    fd.close()

Il fait **12'258 [ko]** et **12 [Mo]**

### Segmenter et tokenizer en une liste de listes de mots, y compris les ponctuations (une liste = une phrase).

In [7]:
#corpus_nettoye = corpus.replace('\r\n','') # on nettoie les retours à la ligne

In [8]:
tokenized = nltk.sent_tokenize(corpus, language="french")

In [9]:
print("Il y a {} phrases dans le corpus".format(len(tokenized)))

Il y a 102216 phrases dans le corpus


In [10]:
tokenized = [nltk.word_tokenize(sentence) for sentence in tokenized]

In [11]:
word_nb = 0
for sentence in tokenized:
    word_nb += len(sentence)

In [12]:
print("Il y a {} mots dans ce corpus".format(word_nb))

Il y a 2335550 mots dans ce corpus


Extraire un fragment (environ 2000 mots) qui servira de donnée de test.

In [13]:
def separate_train_test(sentences):
    
    train_percentage = 0.03

    random.shuffle(sentences)
    
    sentences_length = len(sentences)
    
    train_set_length = int(len(sentences) * train_percentage)
    
    test_set = sentences[ :train_set_length]
    
    train_set = sentences[train_set_length: ]

    return (train_set, test_set)

In [14]:
train_set, test_set = separate_train_test(tokenized)

In [15]:
print("Il y a {} phrases dans le train_set et {} dans le test_set".format(
        len(train_set),
        len(test_set)))

Il y a 99150 phrases dans le train_set et 3066 dans le test_set


## Entraîner un premier modèle de langage de NLTK

En utilisant les instructions disponibles pour le module NLTK LM, entraîner un modèle de langage sur les données d'entraînement. Attention, le package lm n'est disponible qu'à partir de NLTK version 3.4.

In [16]:
from nltk.lm import MLE
import time

Commencer avec un modèle MLE à l'ordre 2, comme montré dans le tutoriel.

In [17]:
order = 2

In [18]:
train, vocab = padded_everygram_pipeline(order, train_set)

In [19]:
lm = MLE(order)

In [20]:
begin = time.time()
lm.fit(train, vocab)
end = time.time()

print("Cela a pris {:.3f} secondes pour entraîner le modèle d'ordre {}".format(
            (end-begin), order))

Cela a pris 66.732 secondes pour entraîner le modèle d'ordre 2


Calculer la perplexité du modèle sur l’ensemble de test.

In [21]:
lm.perplexity(test_set)

inf

Générer quelques phrases dans le style de Balzac selon les explications de NLTK.

In [22]:
for i in range(15):
    print(lm.generate(7))

['administration', 'de', 'deux', '_rubbers_', 'de', 'vanité', ',']
['</s>', 'points', 'à', 'la', 'vieille', 'et', 'qui']
['</s>', 'de', 'salle', 'où', 'il', 'm', "'"]
['était', 'drôlatique', "l'épigramme", 'de', 'rage', ',', 'à']
['par', 'le', 'secret', 'de', 'Béranger', '?', '</s>']
['de', "s'entendre", 'avec', 'lequel', 'il', 'aime', '.']
['voix', '»', 'Je', 'ne', 'donne', 'partout', 'où']
['mon', 'parc', 'situé', 'sur', 'leur', 'effort', 'à']
['ville', ';', 'désormais', 'le', 'chanvre', '.', '</s>']
['mil', '...', '--', 'Quand', ',', 'rien', '.']
['rétabli', 'la', 'poste', 'en', 'pensant', 'que', 'ton']
['ces', 'arabes', "qu'ils", 'analysent', '.', '</s>', 'le']
['<s>', 'Je', 'viens', ',', 'ils', 'seront', 'incalculables']
['manière', "d'être", "l'objet", "d'une", 'fièvre', 'et', 'de']
['des', 'justes', '.', '</s>', ',', 'les', 'fractures']


In [23]:
for i in range(5):
    print(lm.generate(12))

['nombreux', '.', '</s>', 'rapidement', 'que', "l'hiver", ',', 'les', 'flancs', ',', 'la', 'haine']
['à', 'son', 'vieux', 'grognard', 'comme', 'mademoiselle', 'Cormon', ',', 'à', 'son', 'ennemi', 'mortel']
['<s>', 'Dans', 'cet', 'article', 'a', 'donc', 'été', 'donnée', 'à', 'Ursule', ',', 'Gobseck']
[',', 'pour', 'se', 'dit-elle', 'en', 'ouvrant', 'ou', 'ce', 'volume', 'in-18', 'qui', 'puissent']
['les', 'rendre', '...', '»', 'Camusot', 'eut', 'assez', 'politique', 'du', 'boudoir', 'ovale', 'un']


## Entraîner un second modèle de langage de NLTK

En utilisant par exemple le modèle de Laplace.

Le tester aussi sur le corpus de test et comparer les scores.

## Cloze test

Supprimer un mot sur 7 dans le corpus de test.

En utilisant la méthode lm.generate avec du contexte, demander au modèle de langage de prédire ces mots (en utilisant donc 2-4 mots précédents). Quelle est la performance du système ? Comparer les modèles générés en (2) et en (3).

Générez un texte avec les mots en question (1 mot sur 7) remplacés par des blancs, et passez-le à un autre binôme : quelle est leur performance pour deviner les mots ?

Merci d’envoyer votre notebook Jupyter par email au professeur avant le vendredi 14 juin à 23h59.